On Wednesday, 6 March, Chancellor of the Exchequer Jeremy Hunt will present the government's 2024 Spring Budget. Though the specific budgetary provisions Number 11 will put forth remain private, various sources speculate that the chancellor will cut the National Insurance rate, a supplement to his previous cuts in autumn 2023 (analysed by PolicyEngine in [this article](https://policyengine.org/uk/research/autumn-statement-2023)).

In this post, we'll estimate the impacts of such a cut upon the budget, as well as upon society-wide economic performance indicators. We'll also look at the impact these changes would have upon an average UK household.

[Summarize findings]

# Reforms within Spring Budget 2024

News media speculation regarding the Spring Budget's proposed reforms has mooted changes to various different tax provisions. In this article, we will utilise PolicyEngine to model and analyse three:
* Cuts to Employee (Class 1) National Insurance Contributions (NICs) by either 1p or 2p, depending upon the source[^1][^3]
* A 2p cut to the basic rate of tax, lowering the rate from 20p to 18p[^4]
* A fuel duty freeze[^1]

In modelling these reforms, we will assume each took effect on 1 January 2024 and will evaluate their impacts over calendar year 2024. In the future, PolicyEngine hopes to develop the capability to accurately model the true effective date of these sorts of policies.

# NIC Cuts



[^1]:
      [The Guardian, 2024](https://www.theguardian.com/politics/2024/mar/02/tax-and-spending-cuts-will-backfire-economists-warn-jeremy-hunt)
[^2]: 
      [Institute for Government, 2024](https://www.instituteforgovernment.org.uk/comment/five-things-look-out-spring-budget-2024)
[^3]:
      [Financial Times, 2024](https://www.ft.com/content/c2d56d45-c061-4d5a-bf4f-1eddd1fa7d25)
[^4]:
      [Financial Times, 2024](https://www.ft.com/content/1b4f19d7-c7e5-4d16-a7c1-1c12ec0d0592)